# EDA objectives:

1. Clean data:
    * Data type
    * Missing values
    * Duplicates
    * Check outliers

2. Enrich data:
    * Break down the `model` into `make` and `model`

3. Make some plots to show insights of data

In [125]:
import pandas as pd
import streamlit as st
import plotly_express as px

In [126]:
df = pd.read_csv("./../vehicles_us.csv")
print(df.head())

   price  model_year           model  condition  cylinders fuel  odometer   
0   9400      2011.0          bmw x5       good        6.0  gas  145000.0  \
1  25500         NaN      ford f-150       good        6.0  gas   88705.0   
2   5500      2013.0  hyundai sonata   like new        4.0  gas  110000.0   
3   1500      2003.0      ford f-150       fair        8.0  gas       NaN   
4  14900      2017.0    chrysler 200  excellent        4.0  gas   80903.0   

  transmission    type paint_color  is_4wd date_posted  days_listed  
0    automatic     SUV         NaN     1.0  2018-06-23           19  
1    automatic  pickup       white     1.0  2018-10-19           50  
2    automatic   sedan         red     NaN  2019-02-07           79  
3    automatic  pickup         NaN     NaN  2019-03-22            9  
4    automatic   sedan       black     NaN  2019-04-02           28  


# Clean Data

## Missing values

In [127]:
df.isna().sum()

price               0
model_year       3619
model               0
condition           0
cylinders        5260
fuel                0
odometer         7892
transmission        0
type                0
paint_color      9267
is_4wd          25953
date_posted         0
days_listed         0
dtype: int64

Fill missing value strategy:
* `model_year`: median
* `cylinders`: mode
* `odometer`: mean
* `paint_color`: mode
* `is_4wd`: 0 means not a 4 wheels drive

In [128]:
df['model_year'] = df['model_year'].fillna(df['model_year'].mode()[0])
df['cylinders'] = df['cylinders'].fillna(df['cylinders'].median())
df['odometer'] = df['odometer'].fillna(df['odometer'].mean())
df['paint_color'] = df['paint_color'].fillna(df['paint_color'].mode()[0])
df['is_4wd'] = df['is_4wd'].fillna(0.0)
df.isna().sum()

price           0
model_year      0
model           0
condition       0
cylinders       0
fuel            0
odometer        0
transmission    0
type            0
paint_color     0
is_4wd          0
date_posted     0
days_listed     0
dtype: int64

## Data Type

In [129]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    51525 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     51525 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      51525 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   51525 non-null  object 
 10  is_4wd        51525 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


Transform data type strategy:

* Int: `price`, `days_listed`
* Float: `cylinders`, `odometer`,
* Datetime: `model_year`, `date_posted`
* String: `model`, `condition`, `fuel`, `transmission`, `type`, `paint_color`, `is_4wd`

Most of string type columns can be transformed to categorical data. But I won't do it because it not neccessay in this project.

In [130]:
df['is_4wd'].unique()

array([1., 0.])

In [131]:
df['model_year'] = pd.to_datetime(df['model_year'].astype(int).astype(str), format='%Y').dt.year
df['date_posted'] = pd.to_datetime(df['date_posted'])
df['is_4wd'] = df['is_4wd'].apply(lambda x: 'yes' if x == 1.0 else 'no')

## Duplicates

In [132]:
df.duplicated().sum()

0

### Check implicit duplicates

Impcilit duplicate: `model`, `condition`

In [133]:
df.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011,bmw x5,good,6.0,gas,145000.000000,automatic,SUV,white,yes,2018-06-23,19
1,25500,2013,ford f-150,good,6.0,gas,88705.000000,automatic,pickup,white,yes,2018-10-19,50
2,5500,2013,hyundai sonata,like new,4.0,gas,110000.000000,automatic,sedan,red,no,2019-02-07,79
3,1500,2003,ford f-150,fair,8.0,gas,115553.461738,automatic,pickup,white,no,2019-03-22,9
4,14900,2017,chrysler 200,excellent,4.0,gas,80903.000000,automatic,sedan,black,no,2019-04-02,28


In [134]:
df['condition'].unique()

array(['good', 'like new', 'fair', 'excellent', 'salvage', 'new'],
      dtype=object)

In [135]:
print(sorted(df['model'].unique()))

['acura tl', 'bmw x5', 'buick enclave', 'cadillac escalade', 'chevrolet camaro', 'chevrolet camaro lt coupe 2d', 'chevrolet colorado', 'chevrolet corvette', 'chevrolet cruze', 'chevrolet equinox', 'chevrolet impala', 'chevrolet malibu', 'chevrolet silverado', 'chevrolet silverado 1500', 'chevrolet silverado 1500 crew', 'chevrolet silverado 2500hd', 'chevrolet silverado 3500hd', 'chevrolet suburban', 'chevrolet tahoe', 'chevrolet trailblazer', 'chevrolet traverse', 'chrysler 200', 'chrysler 300', 'chrysler town & country', 'dodge charger', 'dodge dakota', 'dodge grand caravan', 'ford econoline', 'ford edge', 'ford escape', 'ford expedition', 'ford explorer', 'ford f-150', 'ford f-250', 'ford f-250 sd', 'ford f-250 super duty', 'ford f-350 sd', 'ford f150', 'ford f150 supercrew cab xlt', 'ford f250', 'ford f250 super duty', 'ford f350', 'ford f350 super duty', 'ford focus', 'ford focus se', 'ford fusion', 'ford fusion se', 'ford mustang', 'ford mustang gt coupe 2d', 'ford ranger', 'ford 

In [136]:
df['type'].unique()

array(['SUV', 'pickup', 'sedan', 'truck', 'coupe', 'van', 'convertible',
       'hatchback', 'wagon', 'mini-van', 'other', 'offroad', 'bus'],
      dtype=object)

The quality of data in columns 'model' and 'condition' is good. There are not many duplicate meaning names, although there are several similar models, e.g., 'ford focus' and 'ford focus se', this is normal and these models should be treated differently. 

I only found 'ford f-150', 'ford f150' and 'ford f-250', 'ford f250'. I will combine them together.

In [137]:
def replace_wrong_values(df, col, replace_dict): 
    for key, value in replace_dict.items(): 
        df[col] = df[col].replace(key, value)
        

replace_dict = {"ford f150": 'ford f-150', 'ford f250': 'ford f-250', 'ford f250 super duty': 'ford f-250 super duty'}

replace_wrong_values(df, 'model',  replace_dict)
print(sorted(df['model'].unique()))

['acura tl', 'bmw x5', 'buick enclave', 'cadillac escalade', 'chevrolet camaro', 'chevrolet camaro lt coupe 2d', 'chevrolet colorado', 'chevrolet corvette', 'chevrolet cruze', 'chevrolet equinox', 'chevrolet impala', 'chevrolet malibu', 'chevrolet silverado', 'chevrolet silverado 1500', 'chevrolet silverado 1500 crew', 'chevrolet silverado 2500hd', 'chevrolet silverado 3500hd', 'chevrolet suburban', 'chevrolet tahoe', 'chevrolet trailblazer', 'chevrolet traverse', 'chrysler 200', 'chrysler 300', 'chrysler town & country', 'dodge charger', 'dodge dakota', 'dodge grand caravan', 'ford econoline', 'ford edge', 'ford escape', 'ford expedition', 'ford explorer', 'ford f-150', 'ford f-250', 'ford f-250 sd', 'ford f-250 super duty', 'ford f-350 sd', 'ford f150 supercrew cab xlt', 'ford f350', 'ford f350 super duty', 'ford focus', 'ford focus se', 'ford fusion', 'ford fusion se', 'ford mustang', 'ford mustang gt coupe 2d', 'ford ranger', 'ford taurus', 'gmc acadia', 'gmc sierra', 'gmc sierra 1

## Split `made` and `model`

In [138]:
def car_make(ori_model):
    model_strings = ori_model.split(" ")
    return(model_strings[0])
    
df['make'] = df['model'].apply(car_make)
df.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make
0,9400,2011,bmw x5,good,6.0,gas,145000.000000,automatic,SUV,white,yes,2018-06-23,19,bmw
1,25500,2013,ford f-150,good,6.0,gas,88705.000000,automatic,pickup,white,yes,2018-10-19,50,ford
2,5500,2013,hyundai sonata,like new,4.0,gas,110000.000000,automatic,sedan,red,no,2019-02-07,79,hyundai
3,1500,2003,ford f-150,fair,8.0,gas,115553.461738,automatic,pickup,white,no,2019-03-22,9,ford
4,14900,2017,chrysler 200,excellent,4.0,gas,80903.000000,automatic,sedan,black,no,2019-04-02,28,chrysler


## Check outliers

In [139]:
df.describe()

,price,model_year,cylinders,odometer,date_posted,days_listed
count,51525.000000,51525.000000,51525.000000,51525.000000,51525,51525.00000
mean,12132.464920,2009.978709,6.112450,115553.461738,2018-10-25 01:57:46.270742528,39.55476
min,1.000000,1908.000000,3.000000,0.000000,2018-05-01 00:00:00,0.00000
25%,5000.000000,2007.000000,4.000000,79181.000000,2018-07-29 00:00:00,19.00000
50%,9000.000000,2011.000000,6.000000,115553.461738,2018-10-25 00:00:00,33.00000
75%,16839.000000,2014.000000,8.000000,146541.000000,2019-01-21 00:00:00,53.00000
max,375000.000000,2019.000000,12.000000,990000.000000,2019-04-19 00:00:00,271.00000
std,10040.803015,6.114081,1.573784,59902.205520,NaN,28.20427


Outliers or wrong numbers:

* `price`: The minimum price is 1 dollar, it looks like a wrong number. I'd like to delete it. The maximum price is 375000, it's ok but worth to double check.
* `model_year`: the oldest car is 1908, seems like a vintage car.
* `odometer`: the minimum value is 0. If there are many zero values, maybe they are new cars. The maximum value is 990000, it looks like a wrong number.

### `price`

In [140]:
df[df['price'] < 10]['price'].count()

802

In [141]:
df[df['price'] > 300000]

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make
12504,375000,1999,nissan frontier,good,6.0,gas,115000.0,automatic,pickup,blue,yes,2018-05-19,21,nissan


In [142]:
df[df['model'] == 'nissan frontier']['price'].median()

10995.0

There are 802 cars which price are less than $10. I think these data will impact data reliability, I will delete all price less than 1000.

The maximum price is from a 'nissan frontier' car. This car is a normal household car rather than some special model. The median price of this model is 10995, so I think it's a wrong value and I'll delete it.

### `model_year`

In [143]:
df[df['model_year'] < 1960]

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make
10018,23900,1955,ford f-250,excellent,6.0,gas,47180.000000,manual,truck,blue,no,2018-12-22,61,ford
14752,15000,1954,ford f-150,excellent,6.0,gas,3565.000000,manual,pickup,black,no,2019-02-16,13,ford
22595,21000,1948,chevrolet impala,like new,8.0,gas,4000.000000,automatic,sedan,red,no,2019-01-18,24,chevrolet
33906,12995,1908,gmc yukon,good,8.0,gas,169328.000000,automatic,SUV,black,no,2018-07-06,34,gmc
33907,12995,1908,cadillac escalade,excellent,8.0,gas,115553.461738,automatic,SUV,white,no,2018-06-24,25,cadillac
34713,5000,1936,ford f-150,excellent,6.0,gas,30000.000000,manual,pickup,purple,no,2018-11-22,10,ford
36582,44900,1949,chevrolet suburban,good,6.0,gas,1800.000000,automatic,wagon,orange,no,2018-08-19,10,chevrolet
39580,35000,1958,chevrolet impala,excellent,8.0,gas,3184.000000,automatic,coupe,black,no,2018-05-19,33,chevrolet
45694,18000,1929,ford f-150,good,8.0,gas,115553.461738,manual,other,silver,no,2018-11-18,59,ford
48414,37900,1958,chevrolet impala,good,8.0,gas,62799.000000,automatic,coupe,white,no,2018-08-11,10,chevrolet


According to the Wikipedia, the first GMC Yukon, Cadillac Escalade, Ford F-150, Chevrolet Impala were introduced in 1991, 1999, 1975, 1958. Therefore many data before 1960 are wrong. I'll delete them.

### `odometer`

In [144]:
df[df['odometer'] > 700000]

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make
2440,5300,2010,toyota corolla,excellent,4.0,gas,840000.0,automatic,sedan,silver,no,2019-02-04,13,toyota
2840,18500,2014,ford explorer,excellent,6.0,gas,866000.0,automatic,SUV,black,yes,2018-07-20,14,ford
5735,7500,2015,hyundai elantra,excellent,4.0,gas,813542.0,other,coupe,brown,no,2018-11-21,5,hyundai
13027,8500,2015,chevrolet malibu,excellent,4.0,gas,838095.0,automatic,sedan,silver,no,2018-05-30,10,chevrolet
14245,6000,1996,ram 2500,good,10.0,gas,920000.0,automatic,truck,blue,no,2019-02-02,43,ram
17816,20000,2011,gmc sierra 1500,good,8.0,gas,790000.0,automatic,pickup,grey,yes,2018-09-08,42,gmc
17869,59900,1964,chevrolet corvette,like new,6.0,gas,990000.0,automatic,convertible,red,no,2018-06-17,28,chevrolet
23839,15000,1998,chevrolet corvette,excellent,8.0,gas,710000.0,automatic,coupe,silver,no,2019-01-29,20,chevrolet
32973,5550,2010,toyota corolla,excellent,6.0,gas,840000.0,automatic,sedan,silver,no,2018-10-21,20,toyota
33578,12995,2014,chrysler 300,like new,6.0,gas,811956.0,automatic,sedan,white,no,2018-11-13,44,chrysler


Those cars run more than 700000 miles but most their condition are 'excellent', 'like new' and 'good'. I think they are wrong number. I'll delete them.

In [145]:
df[df['odometer'] < 10]

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make
352,7997,2009,gmc yukon,excellent,8.0,gas,0.0,automatic,SUV,white,yes,2019-03-16,88,gmc
814,2995,1999,ford f-150,good,6.0,gas,0.0,manual,truck,white,no,2019-03-14,16,ford
1375,5888,2013,toyota 4runner,good,6.0,gas,0.0,automatic,SUV,black,no,2019-02-22,36,toyota
1461,1000,1992,gmc sierra 1500,good,8.0,gas,0.0,automatic,truck,white,no,2018-09-28,47,gmc
1970,10988,2000,ford f-250 sd,good,8.0,diesel,0.0,automatic,truck,white,yes,2018-11-21,39,ford
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49284,3300,2002,hyundai santa fe,good,6.0,gas,0.0,automatic,wagon,white,yes,2019-02-15,7,hyundai
50412,25500,2013,jeep wrangler,good,6.0,gas,0.0,automatic,SUV,yellow,yes,2018-06-12,48,jeep
50631,3600,2012,chrysler 200,good,4.0,gas,0.0,automatic,sedan,red,no,2018-07-24,90,chrysler
50633,1800,2000,ford expedition,fair,8.0,gas,1.0,automatic,SUV,brown,no,2019-01-05,26,ford


There are 246 cars with less than 10 miles but the price is only thousands, I think they are wrong number too. I'll delete them

In [146]:
df['price'] = df.loc[((df['price'] > 1000) & (df['price'] < 300000)), 'price'] 
df['model_year'] = df.loc[(df['model_year'] > 1960), 'model_year']
df['odometer'] = df.loc[((df['odometer'] < 700000) & (df['odometer'] > 10)), 'odometer']
df.describe()

,price,model_year,cylinders,odometer,date_posted,days_listed
count,49963.000000,51512.000000,51525.000000,51258.000000,51525,51525.00000
mean,12489.020135,2009.995147,6.112450,115908.369231,2018-10-25 01:57:46.270742528,39.55476
min,1005.000000,1961.000000,3.000000,13.000000,2018-05-01 00:00:00,0.00000
25%,5500.000000,2007.000000,4.000000,80000.000000,2018-07-29 00:00:00,19.00000
50%,9550.000000,2011.000000,6.000000,115553.461738,2018-10-25 00:00:00,33.00000
75%,16995.000000,2014.000000,8.000000,146969.000000,2019-01-21 00:00:00,53.00000
max,189000.000000,2019.000000,12.000000,660000.000000,2019-04-19 00:00:00,271.00000
std,9755.474114,6.019732,1.573784,58167.944211,NaN,28.20427


# Analysis with plots

## Histogram by car make factories.

In [147]:
model_df = df[['make', 'type']]
model_df.head()

,make,type
0,bmw,SUV
1,ford,pickup
2,hyundai,sedan
3,ford,pickup
4,chrysler,sedan


In [148]:
fig = px.histogram(model_df, x='make', color='type')
fig.show()

In [149]:
df.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make
0,9400.0,2011.0,bmw x5,good,6.0,gas,145000.000000,automatic,SUV,white,yes,2018-06-23,19,bmw
1,25500.0,2013.0,ford f-150,good,6.0,gas,88705.000000,automatic,pickup,white,yes,2018-10-19,50,ford
2,5500.0,2013.0,hyundai sonata,like new,4.0,gas,110000.000000,automatic,sedan,red,no,2019-02-07,79,hyundai
3,1500.0,2003.0,ford f-150,fair,8.0,gas,115553.461738,automatic,pickup,white,no,2019-03-22,9,ford
4,14900.0,2017.0,chrysler 200,excellent,4.0,gas,80903.000000,automatic,sedan,black,no,2019-04-02,28,chrysler


In [150]:
year_price_df = df[['price', 'model_year', 'model']]
fig = px.scatter(year_price_df, x='model_year', y='price')
fig.show()